In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from skorch import NeuralNetClassifier

In [2]:
!pip install /pstore/home/shetabs1/code/iflai/

Defaulting to user installation because normal site-packages is not writeable
Processing /pstore/home/shetabs1/code/iflai
  Created wheel for iflai: filename=iflai-0.0.1-py3-none-any.whl size=23042 sha256=41fa52a5ab05293f6e1e2d8b6eacb96265b480cd3afcb43e95ce521d65da58ef
  Stored in directory: /local/2772878/pip-ephem-wheel-cache-y62kgf3y/wheels/77/c9/46/be8aac478e8f67c427d0996a39b13c1213341ddb146b974964
Successfully built iflai
  Attempting uninstall: iflai
    Found existing installation: iflai 0.0.1
    Uninstalling iflai-0.0.1:
      Successfully uninstalled iflai-0.0.1
You should consider upgrading via the '/pstore/apps/Python/3.8.3-2020.07/bin/python -m pip install --upgrade pip' command.


In [3]:
import warnings
warnings.filterwarnings("ignore")
import os
import torch
from torchvision import transforms
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset

In [4]:
from sklearn.metrics import classification_report

In [6]:
from imblearn.over_sampling import RandomOverSampler
from iflai.dl.util import read_data, get_statistics_h5, calculate_weights
from iflai.dl.dataset import train_validation_test_split_wth_augmentation, Dataset_Generator_Preprocessed_h5
from iflai.dl.models import PretrainedModel

In [7]:
from skorch.callbacks import LRScheduler, Checkpoint
import torch.optim as optim
from skorch.helper import predefined_split

In [8]:
seed_value = 42

os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)

np.random.seed(seed_value)
torch.manual_seed(seed_value)

In [14]:
dataset_name = "synapse_formation"
only_channels = [0,3,4,5,6]
metadata = pd.read_csv("/pstore/home/shetabs1/data/iflai/metadata_subset.csv")
scaling_factor = 4095.
reshape_size = 128
num_channels = len(only_channels)
train_transform = transforms.Compose(
        [transforms.RandomVerticalFlip(),
         transforms.RandomHorizontalFlip(),
         transforms.RandomRotation(45)])
test_transform = transforms.Compose([])
batch_size = 256
num_workers = 2
dev="cuda"

label_map = {'B_cell':0,   
             'T_cell':1,
             'T_cell_with_signaling':2,
             'T_cell_with_B_cell_fragments':3, 
             'B_T_cell_in_one_layer':4,
             'Synapses_without_signaling':5,
            'Synapses_with_signaling':6,
             'No_cell_cell_interaction':7,
             'Multiplets':8, 
        }

In [16]:
indx = metadata.label.isin(['B_cell',   'B_T_cell_in_one_layer',
        'Multiplets', 'Synapses_with_signaling',
       'T_cell_with_B_cell_fragments', 'T_cell',
       'No_cell_cell_interaction', 'Synapses_without_signaling',
         'T_cell_with_signaling'])

metadata = metadata.loc[indx,:]

In [19]:
train_metadata, validation_metadata, test_metadata = train_validation_test_split_wth_augmentation(metadata, metadata["label"])

In [22]:
train_metadata = train_metadata.reset_index(drop = True)
validation_metadata = validation_metadata.reset_index(drop = True)
test_metadata = test_metadata.reset_index(drop = True)

In [24]:
train_dataset = Dataset_Generator_Preprocessed_h5(path_to_data=path_to_data,
                                                      set_indx=train_indx,
                                                      scaling_factor=scaling_factor,
                                                      reshape_size=reshape_size,
                                                      transform=train_transform,
                                                      data_map=data_map,
                                                      only_channels=only_channels,
                                                      num_channels=num_channels)

trainloader = DataLoader(train_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=num_workers)

NameError: name 'path_to_data' is not defined

Help on class Dataset_Generator_Preprocessed_h5 in module iflai.dl.dataset:

class Dataset_Generator_Preprocessed_h5(torch.utils.data.dataset.Dataset)
 |  Dataset_Generator_Preprocessed_h5(*args, **kwds)
 |  
 |  An abstract class representing a :class:`Dataset`.
 |  
 |  All datasets that represent a map from keys to data samples should subclass
 |  it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
 |  data sample for a given key. Subclasses could also optionally overwrite
 |  :meth:`__len__`, which is expected to return the size of the dataset by many
 |  :class:`~torch.utils.data.Sampler` implementations and the default options
 |  of :class:`~torch.utils.data.DataLoader`.
 |  
 |  .. note::
 |    :class:`~torch.utils.data.DataLoader` by default constructs a index
 |    sampler that yields integral indices.  To make it work with a map-style
 |    dataset with non-integral indices/keys, a custom sampler must be provided.
 |  
 |  Method resolution order:
